# Load Store Handler Class

In [ ]:
from linkingtool.hdf5_handler import DataHandler
import linkingtool.visuals as vis

# Define Province Code

In [ ]:
province_code:str='BC'

In [ ]:
store=f"data/store/resources_{province_code}.h5"
datahandler=DataHandler(store)

In [ ]:
cells=datahandler.from_store('cells')

#### Dark Tile Sources

- CartoDark: A dark tile source from CartoDB that offers a dark, visually appealing map. <br>
- CartoMidnight: Another dark-themed tile from CartoDB, designed for night-time visualizations.<br>
- EsriWorldDarkGrayBase: A dark gray base map from Esri, suitable for various applications with a dark theme.<br>
- EsriWorldDarkGrayReference: Similar to the above, this is a reference layer that is also dark gray.<br>
- EsriWorldHillshadeDark: A dark hillshade layer that emphasizes terrain in dark tones.<br>
- EsriOceanBase (context-dependent): While primarily focused on ocean data, it might be rendered in darker tones in certain applications.<br>

#### Non-Dark Tile Sources (for contrast)

These are generally lighter and may not suit a dark-themed visualization: <br>

- CartoEco<br>
- CartoLight<br>
- EsriImagery<br>
- EsriNatGeo<br>
- EsriWorldLightGrayBase<br>
- EsriWorldLightGrayReference<br>
- OSM (OpenStreetMap)<br>
- OpenTopoMap<br>

In [ ]:
import hvplot.pandas
import panel as pn
import param
import holoviews as hv
from holoviews import opts

# Initialize Holoviews extension
hv.extension('bokeh')

cells_c = cells  # Assuming 'cells' is your GeoDataFrame

# Define a dictionary to map columns to specific colormaps
cmap_mapping = {
    'windspeed_gwa': 'viridis',
    'lcoe_wind': 'inferno',
    'Region': 'Category10',
    'CF_IEC2': 'RdYlGn',
    'windspeed_ERA5': 'Purples',
    'nearest_station_distance_km': 'Oranges'
}

# Define the class with a dropdown for column selection
class PlotSelector(param.Parameterized):
    data = param.Selector(objects=list(cmap_mapping.keys()))  # Use keys from the mapping

    @param.depends('data')
    def view(self):
        # Determine the colormap based on the selected column
        selected_cmap = cmap_mapping[self.data]
        
        # Create the base plot with the selected colormap
        plot = cells_c.hvplot(
            color=self.data,  # color by selected column
            cmap=selected_cmap,  # Use the colormap from the mapping
            geo=True, 
            tiles='CartoDark',
            frame_width=1000, 
            data_aspect=.5,
            alpha=0.8,
            line_color='None',
            line_width=0.2,
            hover_line_color='red'
        )

        # Create a title that includes the colormap name
        title = f"{self.data}"
        
        # Overlay the title
        return plot.opts(title=title)

# Instantiate the class and panel
selector = PlotSelector()
pn.Column(selector.param, selector.view).show()


In [ ]:
import hvplot.pandas
import panel as pn
import param
import holoviews as hv
from holoviews import opts

# Initialize Holoviews extension
hv.extension('bokeh')

cells_c = cells  # Assuming 'cells' is your GeoDataFrame

# Define a dictionary to map columns to specific colormaps
cmap_mapping = {
    'windspeed_gwa': 'viridis',
    'lcoe_wind': 'inferno',
    'Region': 'Category10',
    'CF_IEC2': 'RdYlGn',
    'windspeed_ERA5': 'Purples',
    'nearest_station_distance_km': 'Oranges'
}

# Define a dictionary to map base maps to their names
tile_sources = {
    'CartoDark': 'CartoDark',
    'CartoMidnight': 'CartoMidnight',
    'EsriWorldDarkGrayBase': 'EsriWorldDarkGrayBase',
    'EsriWorldDarkGrayReference': 'EsriWorldDarkGrayReference',
    'EsriWorldHillshadeDark': 'EsriWorldHillshadeDark',
    'EsriOceanBase': 'EsriOceanBase'
}

# Define the class with dropdowns for column selection, base map selection, and colormap selection
class PlotSelector(param.Parameterized):
    data = param.Selector(objects=list(cmap_mapping.keys()))  # Use keys from the colormap mapping
    basemap = param.Selector(objects=list(tile_sources.keys()))  # Use keys from tile sources
    colormap = param.Selector(objects=list(cmap_mapping.values()))  # Use values from the colormap mapping

    @param.depends('data', 'basemap', 'colormap')
    def view(self):
        # Get the selected column
        selected_col = self.data
        
        # Determine the colormap based on the selected column
        selected_cmap = self.colormap
        
        # Create the base plot with the selected colormap and base map
        plot = cells_c.hvplot(
            color=selected_col,  # Color by selected column
            cmap=selected_cmap,  # Use the selected colormap
            geo=True, 
            tiles=self.basemap,  # Use the selected tile source
            frame_width=1000, 
            data_aspect=.6,
            alpha=0.8,
            line_color='None',
            line_width=0.2,
            hover_line_color='red',
            # datashade=True,
            save=True  # This will add the save button if rendered correctly
        )

        # Create a title that includes the selected column name and base map
        title = f"{selected_col} using {self.basemap} tiles with {selected_cmap} colormap"
        
        # Overlay the title
        return plot.opts(title=title)

# Instantiate the class and panel
selector = PlotSelector()

# Create a layout with the dropdowns side by side
dropdown_layout = pn.Row(
    pn.Column("### Data Column", selector.param.data, width=200),
    pn.Column("### Base Map", selector.param.basemap, width=200),
    pn.Column("### Colormap", selector.param.colormap, width=200)
)

# Create a final layout with the dropdowns and the plot
layout = pn.Column(dropdown_layout, selector.view, sizing_mode='stretch_both')

# Show the layout
layout.show()


In [ ]:
import hvplot.pandas
import holoviews as hv
from holoviews import opts
from bokeh.layouts import gridplot
from bokeh.io import show

# Initialize Holoviews extension
hv.extension('bokeh')

cells_c = cells  # Assuming 'cells' is your GeoDataFrame

# Define a dictionary to map columns to specific colormaps
cmap_mapping = {
    'windspeed_gwa': 'viridis',
    'lcoe_wind': 'inferno',
    'lcoe_solar': 'inferno',
    'CF_IEC2': 'RdYlGn',
    'CF_IEC3': 'RdYlGn',
    'windspeed_ERA5': 'Purples',
    'nearest_station_distance_km': 'Oranges'
}

# Define a function to create individual plots
def create_plot(column_name, cmap):
    return cells_c.hvplot(
        color=column_name,
        cmap=cmap,
        geo=True,
        tiles='CartoDark',  # Default base map
        frame_width=400,  # Adjust the size of the plots
        data_aspect=.4,
        alpha=0.8,
        line_color='None',
        line_width=0.1,
        hover_line_color='red'
    ).opts(title=column_name,
            show_grid=True,
            show_legend=True,
            tools=['hover', 'pan', 'wheel_zoom','reset','box_select'],
            legend_position='top_right'
        )

# Create a list of plots for each column
plots = [create_plot(col, cmap) for col, cmap in cmap_mapping.items()]

# Create a grid layout for the plots
grid = hv.Layout(plots).cols(2)  # Adjust the number of columns as needed

# Show the layout
hv.save(grid, 'grid_plots.html')  # Save the grid layout as an HTML file


In [ ]:
timeseries_clusters_solar=datahandler.from_store('timeseries_clusters_solar')
timeseries_clusters_wind=datahandler.from_store('timeseries_clusters_wind')
cells=datahandler.from_store('cells')
clusters_solar=datahandler.from_store('clusters_solar')

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Assuming 'cells' is your GeoDataFrame or DataFrame with a 'Region' column
# cells.plot('Region', edgecolor='yellow', linewidth=0.2)

# Create the plot
ax = cells.dissolve(by='Region').plot(edgecolor='none',color='skyblue',alpha=0.5)

# Turn off the grid
ax.grid(False)

# Save the figure as an SVG file
plt.savefig('plot2.svg', format='svg')

# Optionally, you can also close the plot if you don't want to display it
plt.close()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Assuming 'cells' is your GeoDataFrame or DataFrame with a 'Region' column
# cells.plot('Region', edgecolor='yellow', linewidth=0.2)

# Create the plot
ax = cells.plot('Region', edgecolor='yellow', linewidth=0.2)

# Turn off the grid
ax.grid(False)

# Save the figure as an SVG file
plt.savefig('plot.svg', format='svg')

# Optionally, you can also close the plot if you don't want to display it
plt.close()


In [ ]:
import pandas as pd
import plotly.graph_objects as go

# Assuming 'df' is your DataFrame containing time series data
df = timeseries_clusters_solar  # Load your DataFrame

# Define a list of colors (you can customize this list as needed)
colors = ['blue', 'orange', 'green', 'red', 'purple', 'brown', 'pink', 'gray']  # Add more colors if needed

# Create a list of traces, one for each column
traces = []
for i, col in enumerate(df.columns):
    trace = go.Scatter(
        x=df.index,
        y=df[col],
        mode='lines',
        name=col,
        visible=False,
        line=dict(color=colors[i % len(colors)])  # Cycle through colors
    )
    traces.append(trace)

# Make the first column visible initially
traces[0]['visible'] = True

# Create dropdown options for each column
dropdown_buttons = []
for i, col in enumerate(df.columns):
    button = dict(
        args=[{'visible': [False] * len(df.columns)}],  # Make all traces invisible first
        label=col,
        method='update'
    )
    button['args'][0]['visible'][i] = True  # Make only the selected trace visible
    dropdown_buttons.append(button)

# Define the layout for the figure
layout = go.Layout(
    title="Time Series - Solar",
    xaxis_title="DateTime",
    yaxis_title="Value",
    updatemenus=[{
        'buttons': dropdown_buttons,
        'direction': 'down',
        'showactive': True
    }]
)

# Create the figure with all traces and layout
fig = go.Figure(data=traces, layout=layout)

# Show the figure
fig.show()


In [ ]:
import pandas as pd
import plotly.graph_objects as go

# Assuming 'df' is your DataFrame containing time series data
df = timeseries_clusters_wind  # Load your DataFrame

# Define a list of colors (you can customize this list as needed)
colors = ['blue', 'orange', 'green', 'red', 'purple', 'brown', 'pink', 'gray']  # Add more colors if needed

# Create a list of traces, one for each column
traces = []
for i, col in enumerate(df.columns):
    trace = go.Scatter(
        x=df.index,
        y=df[col],
        mode='lines',
        name=col,
        visible=False,
        line=dict(color=colors[i % len(colors)])  # Cycle through colors
    )
    traces.append(trace)

# Make the first column visible initially
traces[0]['visible'] = True

# Create dropdown options for each column
dropdown_buttons = []
for i, col in enumerate(df.columns):
    button = dict(
        args=[{'visible': [False] * len(df.columns)}],  # Make all traces invisible first
        label=col,
        method='update'
    )
    button['args'][0]['visible'][i] = True  # Make only the selected trace visible
    dropdown_buttons.append(button)

# Define the layout for the figure
layout = go.Layout(
    title="Time Series- Wind",
    xaxis_title="DateTime",
    yaxis_title="Value",
    updatemenus=[{
        'buttons': dropdown_buttons,
        'direction': 'down',
        'showactive': True
    }]
)

# Create the figure with all traces and layout
fig = go.Figure(data=traces, layout=layout)

# Show the figure
fig.show()


In [ ]:
# data1=solar.get_CF_timeseries()
# data2=solar.get_cell_capacity()
# data3=solar.find_grid_nodes()
# data5=solar.score_cells()
# data=datahandler.from_store('cells') 
vis.get_selected_vs_missed_visuals(cells, 
                              'BC',
                              'solar',
                              lcoe_threshold=20, 
                              CF_threshold=0.15, 
                              capacity_threshold=1,
                              text_box_x=.65,
                              text_box_y=.965,
                              title_x=.65,
                              title_y=1,
                              font_size=8,
                              dpi=400,
                              figsize=(8,6),
                              save=True
                              )

In [ ]:
lcoe_threshold=20
wcss_tolr=0.05
filtered_cells=cells[cells['lcoe_solar'] <= lcoe_threshold]
filtered_clusters=clusters_solar[clusters_solar['lcoe_solar'] <= lcoe_threshold]

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt

# Define the LCOE threshold
lcoe_threshold = 20  # You can set this to the appropriate value as needed

# Create a figure with 1 row and 2 columns
fig, ax = plt.subplots(1, 2, figsize=(12, 6))

# Plot the selected cells on the left
filtered_cells.plot(
    column='lcoe_solar',
    ax=ax[0],
    edgecolor='white',
    linewidth=0.2,
    legend=True,
    legend_kwds={'label': "Cells LCOE ($/kWh)", 'orientation': "vertical"}
)
ax[0].set_title('Cells')
ax[0].set_xlabel('Longitude')
ax[0].set_ylabel('Latitude')

# Plot the filtered clusters on the right
filtered_clusters.plot(
    column='lcoe_solar',
    ax=ax[1],
    edgecolor='white',
    linewidth=0.2,
    legend=True,
    # legend_kwds={'label': "Cluster LCOE ($/kWh)", 'orientation': "vertical"}
)
ax[1].set_title(f'Representative Clusters [@ WCSS tolerance {wcss_tolr}')
ax[1].set_xlabel('Longitude')
ax[1].set_ylabel('Latitude')

# Set a common title for the entire figure
fig.suptitle(f"BC- Selected Cells vs their Clusters \n[@ lcoe >={lcoe_threshold} $/kWh]", fontsize=16, fontweight='bold')

# Adjust layout
plt.tight_layout()
plt.subplots_adjust(top=0.85)  # Adjusts the top space to make room for the main title
plt.savefig(f'vis/linking/solar/cells vs cluster_ALL REGIONS_BC')
# Show the plots
plt.show()


In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

# Get unique regions from the filtered clusters
unique_regions = filtered_clusters['Region'].unique()

# Loop through each unique region and create plots
for region in unique_regions:
    # Filter clusters and cells based on the current region
    region_filtered_clusters = filtered_clusters[filtered_clusters['Region'] == region]
    total_clusters = max(region_filtered_clusters['Cluster_No'])

    # Prepare a list to collect all dissolved cell indices for the region
    dissolved_cell_indices = []

    # Collect dissolved indices for the current region
    for _, row in region_filtered_clusters.iterrows():
        dissolved_cell_indices.extend(list(dissolved_indices[row.Region][row.Cluster_No]))

    # Extract the dissolved cells based on the collected indices
    dissolved_cells_region = cells.loc[dissolved_cell_indices]
    total_cells = len(dissolved_cells_region)

    # Create a figure with a single row and two columns
    fig, ax = plt.subplots(1, 2, figsize=(14, 7))

    # Plot all filtered clusters for the current region on the left
    cmap = "plasma"  # Changed colormap for better visibility
    region_filtered_clusters.plot(
        column='lcoe',
        edgecolor='gray',
        linewidth=0.8,
        ax=ax[0],
        cmap=cmap,
        legend=False  # Disable the default legend
    )
    
    # Create a colorbar
    sm = plt.cm.ScalarMappable(cmap=cmap, norm=plt.Normalize(vmin=region_filtered_clusters['lcoe'].min(), vmax=region_filtered_clusters['lcoe'].max()))
    sm.set_array([])
    cbar = fig.colorbar(sm, ax=ax[0], orientation='vertical')
    cbar.set_label("Filtered Clusters' LCOE (kWh)")

    ax[0].set_title(f'Filtered Clusters ({total_clusters}) for Region: {region}')
    ax[0].set_xlabel('Longitude')
    ax[0].set_ylabel('Latitude')

    # Plot all dissolved cells for the current region on the right
    dissolved_cells_region.plot(
        column='lcoe',
        edgecolor='grey',
        ax=ax[1],
        linewidth=0.6,
        alpha=0.7,  # Slightly transparent for better visibility
        cmap=cmap,
        legend=False
    )

    # Create a colorbar for the right plot
    sm_right = plt.cm.ScalarMappable(cmap=cmap, norm=plt.Normalize(vmin=dissolved_cells_region['lcoe'].min(), vmax=dissolved_cells_region['lcoe'].max()))
    sm_right.set_array([])
    cbar_right = fig.colorbar(sm_right, ax=ax[1], orientation='vertical')
    cbar_right.set_label("Dissolved Cells LCOE (kWh)")

    ax[1].set_title(f'Dissolved Cells ({total_cells}) for Region: {region}')
    ax[1].set_xlabel('Longitude')
    ax[1].set_ylabel('Latitude')

    # Set a common title for the figure
    fig.suptitle(f"Cells vs Representative Clusters (@ WCSS tolerance ={wcss_tolr}) \n {region}, {select} [@ LCOE <= {lcoe_threshold} $/kWh]", fontsize=16)

    # Adjust layout
    plt.tight_layout()
    plt.savefig(f'vis/Solar/cells_vs_cluster_{region}.png')  # Added .png extension for clarity
    plt.show()


In [ ]:
lcoe_threshold=20
wcss_tolr=0.05
filtered_cells=cells[cells['lcoe'] <= lcoe_threshold]
filtered_clusters=all_clusters[all_clusters['lcoe'] <= lcoe_threshold]

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt

# Define the LCOE threshold
lcoe_threshold = 20  # You can set this to the appropriate value as needed

# Create a figure with 1 row and 2 columns
fig, ax = plt.subplots(1, 2, figsize=(12, 6))

# Plot the selected cells on the left
filtered_cells.plot(
    column='lcoe',
    ax=ax[0],
    edgecolor='white',
    linewidth=0.2,
    legend=True,
    legend_kwds={'label': "Cells LCOE ($/kWh)", 'orientation': "vertical"}
)
ax[0].set_title('Cells')
ax[0].set_xlabel('Longitude')
ax[0].set_ylabel('Latitude')

# Plot the filtered clusters on the right
filtered_clusters.plot(
    column='lcoe',
    ax=ax[1],
    edgecolor='white',
    linewidth=0.2,
    legend=True,
    # legend_kwds={'label': "Cluster LCOE ($/kWh)", 'orientation': "vertical"}
)
ax[1].set_title(f'Representative Clusters [@ WCSS tolerance {wcss_tolr}')
ax[1].set_xlabel('Longitude')
ax[1].set_ylabel('Latitude')

# Set a common title for the entire figure
fig.suptitle(f"{select} - Selected Cells vs their Clusters \n[@ lcoe >={lcoe_threshold} $/kWh]", fontsize=16, fontweight='bold')

# Adjust layout
plt.tight_layout()
plt.subplots_adjust(top=0.85)  # Adjusts the top space to make room for the main title
plt.savefig(f'vis/Solar/cells vs cluster_ALL REGIONS_{select}')
# Show the plots
plt.show()


In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

# Get unique regions from the filtered clusters
unique_regions = filtered_clusters['Region'].unique()

# Loop through each unique region and create plots
for region in unique_regions:
    # Filter clusters and cells based on the current region
    region_filtered_clusters = filtered_clusters[filtered_clusters['Region'] == region]
    total_clusters = max(region_filtered_clusters['Cluster_No'])

    # Prepare a list to collect all dissolved cell indices for the region
    dissolved_cell_indices = []

    # Collect dissolved indices for the current region
    for _, row in region_filtered_clusters.iterrows():
        dissolved_cell_indices.extend(list(solar.solar_clusters_nt.dissolved_indices[row.Region][row.Cluster_No]))

    # Extract the dissolved cells based on the collected indices
    dissolved_cells_region = cells.loc[dissolved_cell_indices]
    total_cells = len(dissolved_cells_region)

    # Create a figure with a single row and two columns
    fig, ax = plt.subplots(1, 2, figsize=(14, 7))

    # Plot all filtered clusters for the current region on the left
    cmap = "plasma"  # Changed colormap for better visibility
    region_filtered_clusters.plot(
        column='lcoe',
        edgecolor='gray',
        linewidth=0.8,
        ax=ax[0],
        cmap=cmap,
        legend=False  # Disable the default legend
    )
    
    # Create a colorbar
    sm = plt.cm.ScalarMappable(cmap=cmap, norm=plt.Normalize(vmin=region_filtered_clusters['lcoe'].min(), vmax=region_filtered_clusters['lcoe'].max()))
    sm.set_array([])
    cbar = fig.colorbar(sm, ax=ax[0], orientation='vertical')
    cbar.set_label("Filtered Clusters' LCOE (kWh)")

    ax[0].set_title(f'Filtered Clusters ({total_clusters}) for Region: {region}')
    ax[0].set_xlabel('Longitude')
    ax[0].set_ylabel('Latitude')

    # Plot all dissolved cells for the current region on the right
    dissolved_cells_region.plot(
        column='lcoe',
        edgecolor='grey',
        ax=ax[1],
        linewidth=0.6,
        alpha=0.7,  # Slightly transparent for better visibility
        cmap=cmap,
        legend=False
    )

    # Create a colorbar for the right plot
    sm_right = plt.cm.ScalarMappable(cmap=cmap, norm=plt.Normalize(vmin=dissolved_cells_region['lcoe'].min(), vmax=dissolved_cells_region['lcoe'].max()))
    sm_right.set_array([])
    cbar_right = fig.colorbar(sm_right, ax=ax[1], orientation='vertical')
    cbar_right.set_label("Dissolved Cells LCOE (kWh)")

    ax[1].set_title(f'Dissolved Cells ({total_cells}) for Region: {region}')
    ax[1].set_xlabel('Longitude')
    ax[1].set_ylabel('Latitude')

    # Set a common title for the figure
    fig.suptitle(f"Cells vs Representative Clusters (@ WCSS tolerance ={wcss_tolr}) \n {region}, {select} [@ LCOE <= {lcoe_threshold} $/kWh]", fontsize=16)

    # Adjust layout
    plt.tight_layout()
    plt.savefig(f'vis/Solar/cells_vs_cluster_{region}.png')  # Added .png extension for clarity
    plt.show()


In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

# Get unique regions from the filtered clusters
unique_regions = filtered_clusters['Region'].unique()

# Loop through each unique region and create plots
for region in unique_regions:
    # Filter clusters and cells based on the current region
    region_filtered_clusters = filtered_clusters[filtered_clusters['Region'] == region]
    total_clusters=max(region_filtered_clusters['Cluster_No'])

    # Prepare a list to collect all dissolved cell indices for the region
    dissolved_cell_indices = []

    # Collect dissolved indices for the current region
    for _, row in region_filtered_clusters.iterrows():
        dissolved_cell_indices.extend(list(solar.solar_clusters_nt.dissolved_indices[row.Region][row.Cluster_No]))

    # Extract the dissolved cells based on the collected indices
    dissolved_cells_region = cells.loc[dissolved_cell_indices]
    total_cells=len(dissolved_cells_region)

    # Create a figure with 1 row and 2 columns
    fig, ax = plt.subplots(1, 2, figsize=(12, 6))

    # Plot all filtered clusters for the current region on the left
    cmap_left = "viridis"  # Specify your desired colormap here
    region_filtered_clusters.plot(
        column='lcoe',
        edgecolor='gray',
        linewidth=0.8,
        ls=":",
        ax=ax[0],
        cmap=cmap_left,  # Use the specified colormap
        legend=False  # Disable the default legend
    )

    # Create a divider for the left plot to add colorbar
    divider_left = make_axes_locatable(ax[0])
    cax_left = divider_left.append_axes("right", size="5%", pad=0.1)  # Adjust size and padding as needed

    # Create a colorbar
    sm_left = plt.cm.ScalarMappable(cmap=cmap_left, norm=plt.Normalize(vmin=region_filtered_clusters['lcoe'].min(), vmax=region_filtered_clusters['lcoe'].max()))
    sm_left.set_array([])
    cbar_left = fig.colorbar(sm_left, cax=cax_left)
    cbar_left.set_label("Filtered Clusters' LCOE (kWh)")

    ax[0].set_title(f'Filtered Clusters ({total_clusters}) for Region: {region}')
    ax[0].set_xlabel('Longitude')
    ax[0].set_ylabel('Latitude')

    # Plot all dissolved cells for the current region on the right
    cmap_right = "viridis"  # Specify your desired colormap here
    dissolved_cells_region.plot(
        column='lcoe',
        edgecolor='grey',
        ax=ax[1],
        linewidth=0.6,
        ls=":",
        alpha=1,  # Adjust transparency for better visualization
        cmap=cmap_right,  # Use the specified colormap
        legend=False  # Disable the default legend
    )

    # Create a divider for the right plot to add colorbar
    divider_right = make_axes_locatable(ax[1])
    cax_right = divider_right.append_axes("right", size="5%", pad=0.1)  # Adjust size and padding as needed

    # Create a colorbar
    sm_right = plt.cm.ScalarMappable(cmap=cmap_right, norm=plt.Normalize(vmin=dissolved_cells_region['lcoe'].min(), vmax=dissolved_cells_region['lcoe'].max()))
    sm_right.set_array([])
    cbar_right = fig.colorbar(sm_right, cax=cax_right)
    cbar_right.set_label("Dissolved Cells LCOE (kWh)")

    ax[1].set_title(f'Dissolved Cells ({total_cells}) for Region: {region}')
    ax[1].set_xlabel('Longitude')
    ax[1].set_ylabel('Latitude')

    # Set a common title for the figure
    fig.suptitle(f"Cells vs Representative Clusters ( @ WCSS tolerance ={wcss_tolr}) \n {region},{select} [@ LCOE <= {lcoe_threshold} $/kWh]", fontsize=16)

    # Adjust layout
    plt.tight_layout()
    plt.savefig(f'vis/Solar/cells vs cluster_{region}')
    # Show the plots for the current region
    plt.show()
